### GPU Setup


for use local gpu run this comand in anaconda prompet:

      1) pip install jupyter_http_over_ws
      2) jupyter serverextension enable --py jupyter_http_over_ws
      3) jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0
      
copy url and paste on colab

In [ ]:
# import torch
#Check GPU
# torch.cuda.get_device_name(0)

# article

In [ ]:

# from PyPDF2 import PdfReader

# pdf_reader = PdfReader("Is ChatGPT a Good Recommender A Preliminary Study.pdf")
# pdf_text = ''
# for page_num in range(len(pdf_reader.pages)):
#     page = pdf_reader.pages[page_num]
#     pdf_text += page.extract_text()

# pdf_text[401:-15625]

# file_path = 'example.txt'
# with open(file_path, 'w') as file:
#     # 2. Write the string to the file
#     string_to_save = pdf_text[401:-15625]
#     file.write(string_to_save)

This structure indicates that the LLMRec system can handle scenarios where either some input data is provided or where only an instruction is needed. The system then uses these templates to format the prompts sent to the language model, and the responses are extracted based on the specified split marker.

# readme

## overview of the project:

LLMRec is a recommender system based on large language models (LLMs), designed for benchmarking these models on various recommendation tasks. The key features and components :

**Abstract**: LLMRec focuses on five recommendation tasks: rating prediction, sequential recommendation, direct recommendation, explanation generation, and review summarization. It utilizes well-designed templates to transform recommendation data into textual prompts for benchmarking popular LLMs.

**Prompt Construction**: The project involves using various prompts for evaluating `*off-the-shelf*` LLMs. Additionally, there's mention of task-specific finetuning for various open-source LLMs, supported by LLMRec's prompt construction module.

**Data Acquisition**: `?`

Given this overview, the project focuses on using LLMs for various recommendation tasks, with an emphasis on prompt construction and evaluation.

# utils

several utility functions for data processing, including filtering and converting JSON data, counting words in sentences, splitting and merging CSV files, and comparing CSV files.

# data format transform `func`

In [ ]:
import json
import os
import argparse
import pandas as pd

In [ ]:
def convert_json_data(input_file, data_type='generic', target_id=None):
  """
  Processes and converts JSON data based on specified criteria and saves it as a new JSON file.

  This function can handle both generic and recommendation-specific data. It filters and processes
  entries in the input JSON file based on the specified criteria and saves the processed data to a new file.
  The output file's name is derived from the input file's name and the provided criteria.

  Parameters:
  - input_file (str): Path to the input JSON file.
  - data_type (str): Type of data to process ('generic' or 'recommendation').
  - target_id (str, optional): ID for filtering data. Used only for recommendation data.
                              If None, no filtering is applied.
  """

  # Initialize an empty list to store filtered data
  filtered_data = []

  # Construct the base of the output file name by removing the extension from the input file
  output_file_name = os.path.splitext(input_file)[0]

  # Determine the suffix for the output file based on data type and target ID
  output_suffix = f"_{data_type}" + (f"_{target_id}" if target_id else '_all')

  # Open and read the input file
  with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
      # Load each line as a JSON object
      data = json.loads(line)

      # Process recommendation data if specified
      if data_type == 'recommendation' and (target_id is None or target_id == 'all' or data['task_type'] == target_id):
        # Append the transformed data to the filtered_data list
        filtered_data.append({
          'instruction': data['source'],
          'input': '',
          'output': data['target']
        })
      elif data_type == 'generic':
        # For generic data, append the data as is (or apply any generic transformation if needed)
        filtered_data.append(data)

  # Construct the full output file name
  output_file = output_file_name + output_suffix + '.json'

  # Write the filtered and processed data to the output file
  with open(output_file, 'w', encoding='utf-8') as f:
    for data in filtered_data:
      # Write each piece of data as a JSON object, followed by a newline
      f.write(json.dumps(data, ensure_ascii=False) + '\n')

  # Print a confirmation message with the name of the output file
  print(f"Processed file saved as: {output_file}")


In [ ]:
def word_count(input_file):
  """
  Calculates the maximum word count in sentences from a JSON file.

  This function reads each line (sentence) from the specified JSON file and calculates the word count.
  It keeps track of and prints the maximum word count found in the file. This is useful for analyzing
  the length of text data.

  Parameters:
  - input_file (str): Path to the input JSON file.
  """
  # Calculate the maximum word count in sentences from a JSON file
  max_word_count = 0
  with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
      data = json.loads(line)
      source_sentence = data['source']
      # Count the number of words in the sentence (split by spaces)
      word_count = len(source_sentence.split())
      # Update the maximum word count if current sentence has more words
      max_word_count = max(max_word_count, word_count)

  print(f"Maximum word count in a sentence in the file: {max_word_count}")

In [ ]:
def split_csv_file(file_path, num_parts):
  """
  Splits a CSV file into multiple parts.

  The function divides a CSV file into the specified number of parts and saves each part as a separate
  CSV file. This is useful for handling large datasets or creating data splits for model training and testing.

  Parameters:
  - file_path (str): Path to the CSV file to be split.
  - num_parts (int): Number of parts to split the file into.
  """
  # Read the CSV file using pandas
  df = pd.read_csv(file_path)

  # Calculate the size of each part of the split
  total_rows = len(df)
  rows_per_part = total_rows // num_parts

  # Split the data and save as multiple CSV files
  file_name = os.path.basename(file_path)
  file_name_without_ext = os.path.splitext(file_name)[0]
  file_directory = os.path.dirname(file_path)

  for i in range(num_parts):
    start_index = i * rows_per_part
    end_index = start_index + rows_per_part
    # For the last part, include all remaining rows
    if i == num_parts - 1:
      end_index = total_rows

    part_df = df[start_index:end_index]
    part_file_name = f"{file_name_without_ext}_{i + 1}.csv"
    part_file_path = os.path.join(file_directory, part_file_name)
    part_df.to_csv(part_file_path, index=False)


In [ ]:
def merge_csv_files(file_name, output_file):
  """
  Merges multiple CSV files into a single file.

  This function finds all CSV files in the specified directory that start with the given file name
  and merges them into a single CSV file. The merged file is saved to the specified output path.

  Parameters:
  - file_name (str): Base name of the files to be merged.
  - output_file (str): Path for the output merged CSV file.
  """
  # Locate all CSV files in the directory that match the specified file name
  file_directory = os.path.dirname(output_file)
  csv_files = [file for file in os.listdir(file_directory) if file.startswith(file_name) and file.endswith('.csv') and file != file_name + '.csv']
  csv_files.sort()  # Sort the files by name

  # Concatenate all found CSV files into a single DataFrame and save it
  merged_df = pd.concat([pd.read_csv(os.path.join(file_directory, file)) for file in csv_files])
  merged_df.to_csv(output_file, index=False)


In [ ]:
def compare_csv_files(file1, file2):
  """
  Compares two CSV files to check if they are identical.

  The function reads two CSV files and compares them. It prints a message indicating whether
  the files are completely identical or not.

  Parameters:
  - file1 (str): Path to the first CSV file.
  - file2 (str): Path to the second CSV file.
  """
  # Compare two CSV files to check if they are identical
  df1 = pd.read_csv(file1)
  df2 = pd.read_csv(file2)

  # Print a message based on whether the DataFrames are identical
  if df1.equals(df2):
    print("The two CSV files are identical.")
  else:
    print("The two CSV files are not identical.")

In [ ]:
if __name__ == '__main__':
  # Example usage and testing code
  # Uncomment and modify according to the specific use case
  parser = argparse.ArgumentParser(description='Filter and convert JSON file.')
  parser.add_argument('input_file', type=str, help='Path to the input JSON file', default='data/train_prompt.json')
  parser.add_argument('target_id', type=str, nargs='?', default=None, help='Target ID for filtering (optional)')
  args = parser.parse_args()

  input_file_path = args.input_file
  target_id = args.target_id

  filter_and_convert(input_file_path, target_id)
  file_path = 'data/test.csv'
  num_parts = 3
  split_csv_file(file_path, num_parts)
  merge_csv_files('test', 'data/combine_test.csv')
  file1 = 'data/test.csv'
  file2 = 'data/combine_test.csv'
  compare_csv_files(file1, file2)
  word_count('data/chatglm-data-full/full_train.json')

# Templates `JSON`

In [ ]:
Template = {
    "description": "Template for LLMRec",
    "prompt_input": "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "### Instruction:\n{instruction}\n\n### Response:\n",
    "response_split": "### Response:"
}


The `rec_template.json` file contains the structure for the prompt templates used in the LLMRec system. Here is a breakdown of its contents:

1. **Description**: It is labeled as 'Template for LLMRec', indicating its purpose in the system.

2. **Prompt with Input (`prompt_input`)**:
   - This template is used when there is an input provided for the prompt.
   - The format includes an 'Instruction' section, an 'Input' section, and a placeholder for 'Response'.
   - The placeholders `{instruction}` and `{input}` will be replaced with actual instruction and input data when generating a prompt.

3. **Prompt without Input (`prompt_no_input`)**:
   - This template is used when there is no input data to include in the prompt.
   - It only includes an 'Instruction' section followed by a placeholder for 'Response'.

4. **Response Split (`response_split`)**:
   - This is a marker used to split the generated text to extract the 'Response' part.
   - It is set to '### Response:', which means this marker is used to identify where the response begins in the model's output.


# Prompter `class`

The design of the `Prompter` class indicates a flexible approach to generating prompts for recommendation tasks, where the specifics of each prompt are determined by a `JSON` template. This aligns with the concept of transforming recommendation tasks into natural language tasks, as described in the article.

In [ ]:
import json
import os.path as osp
from typing import Union

class Prompter(object):

    # Slots for memory efficiency (instances of this class can only have these two attributes, and no other attributes can be added dynamically.)
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        """
        Initialize the Prompter with a specific template.

        Parameters:
        - template_name (str): The name of the template file to use for generating prompts.
        - verbose (bool): If True, the prompter will print additional information.

        Returns:
        None
        """

        self._verbose = verbose

        # Set a default template if not provided
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        # Load the template file
        file_name = osp.join("./templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")

        with open(file_name) as fp:
            self.template = json.load(fp)

        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(self, instruction: str, input: Union[None, str] = None, label: Union[None, str] = None,) -> str:
        ''''
        Generate a prompt based on the provided instruction and optional input/label

        Parameters:
        - instruction (str): Instruction or context for the prompt.
        - input (Union[None, str]): Optional input data to include in the prompt.
        - label (Union[None, str]): Optional label or expected output to append.

        Returns:
        str: The generated prompt.
        '''

        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )

        # Append label (response/output) if provided
        if label:
            res = f"{res}{label}"

        # Print the prompt if verbose mode is on
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        """
        Extract the response part from the model's output.

        Parameters:
        - output (str): The complete output text from the model.

        Returns:
        str: The extracted response portion of the output.
        """
        return output.split(self.template["response_split"])[1].strip()


# generation

This will involve extracting the core functionality needed for generating prompts and obtaining model outputs, and then restructuring it for interactive use in a Jupyter notebook.

In [ ]:
! pip install peft

In [ ]:
import sys
import torch
import transformers
from peft import PeftModel # A model wrapper for performance improvements
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

# from utils.prompter import Promper # Custom class for managing prompt templates

### load model
Matches and sets up the device for the model, and loads the model accordingly.

In [ ]:
def load_model(base_model: str = "", load_8bit: bool = False, lora_weights: str = "tloen/alpaca-lora-7b"):
  """
  Matches and sets up the device for the model, and loads the model accordingly and config it.

  Parameters:
  - base_model (str): The base model identifier.
  - load_8bit (bool): Whether to load the model in 8-bit precision for better performance.
  - lora_weights (str): Path or identifier for LoRA weights for the model.

  Returns:
  The loaded model, set up for the appropriate device.
  """
  # Ensure that a base model is specified
  assert (
      base_model
  ), "Please specify a --base_model, e.g. --base_model='decapoda-research/llama-7b-hf'"

  # Determine the device: CUDA, MPS, or CPU
  device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

  # Common parameters for model loading
  common_params = {
    "base_model": base_model,
    "torch_dtype": torch.float16 if device != "cpu" else None,
    "device_map": {"": device}
  }

  # Load the base model with specific settings based on the device
  match device:
    case "cuda":
      common_params.update({"load_in_8bit": load_8bit})
    case "mps":
      pass  # No additional parameters for MPS
    case _:
      common_params.update({"low_cpu_mem_usage": True})

  model = LlamaForCausalLM.from_pretrained(**common_params)
  # Load PeftModel with LoRA weights
  model = PeftModel.from_pretrained(model, lora_weights, **common_params)

  # Unwind broken configurations from a specific source, possibly due to model incompatibilities or custom requirements
  model.config.pad_token_id = tokenizer.pad_token_id = 0  # Set padding token ID to 0 (typically unknown token)
  model.config.bos_token_id = 1  # Set beginning of sentence token ID
  model.config.eos_token_id = 2  # Set end of sentence token ID

  # Convert the model to half precision for improved performance, unless 8-bit loading is specified
  if not load_8bit:
    model.half()  # seems to fix bugs for some users.

  # Set the model to evaluation mode
  model.eval()

  # Compile the model with TorchScript if using PyTorch version 2 or above and not on Windows platform
  if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)
  return model


### generate
Main function to set up and run the language model for generating outputs based on prompts

In [ ]:
# Define the evaluate function to generate responses based on instructions and input
def generate(base_model, instruction, input=None, prompt_template: str = "med_template",
             temperature=0.1, top_p=0.75, top_k=40, num_beams=4, max_new_tokens=128, **kwargs,):
  """
  Generate a response from the model based on the given instruction and optional input.

  This function takes an instruction (and optionally additional input), generates a prompt,
  and then uses the model to generate a response based on this prompt. The function allows
  configuration of various generation parameters like temperature, top_p, top_k, and num_beams.

  Parameters:
  - base_model (str): The base model for language generation to be used.
  - instruction (str): The instruction or query to be processed by the model.
  - input (str, optional): Additional input data to be included in the prompt.
  - temperature (float): The temperature for controlling randomness in response generation.
  - top_p (float): Nucleus sampling parameter controlling the size of the probability mass considered for sampling.
  - top_k (int): Top-k sampling parameter controlling the number of highest probability vocabulary tokens to keep for sampling.
  - num_beams (int): Number of beams for beam search. More beams increase the chances of finding more diverse solutions but are slower.
  - max_new_tokens (int): The maximum number of new tokens to generate in the response.
  - prompt_template (str): Name of the prompt template to use.
  - **kwargs: Additional keyword arguments for generation configuration.

  Returns:
  str: The generated response from the model.
  """
  # Initialize the prompter with the specified template
  prompter = Prompter(prompt_template)

  # Load the tokenizer for the specified base model
  tokenizer = LlamaTokenizer.from_pretrained(base_model)

  # Generate the prompt using the provided instruction and input
  prompt = prompter.generate_prompt(instruction, input)

  # Tokenize the prompt for model input
  inputs = tokenizer(prompt, return_tensors="pt")

  input_ids = inputs["input_ids"].to(device)

  # Configure generation parameters (temperature, top_p, etc.)
  generation_config = GenerationConfig(
    temperature=temperature,
    top_p=top_p,
    top_k=top_k,
    num_beams=num_beams,
    **kwargs,
  )

  # Load the model based on the device and settings
  model = load_model(base_model=base_model)

  # Generate the output without computing gradients (for efficiency)
  with torch.no_grad():
    generation_output = model.generate(
      input_ids=input_ids,
      generation_config=generation_config,
      return_dict_in_generate=True,
      output_scores=True,
      max_new_tokens=max_new_tokens,
    )

  # Decode the generated sequence to human-readable text
  s = generation_output.sequences[0]
  output = tokenizer.decode(s)
  # Extract and return the response from the output
  return prompter.get_response(output)


In [ ]:
# Example usage
base_model = "decapoda-research/llama-7b-hf"
prompt_template = "med_template"
generation_params = {
    "temperature": 0.1,
    "top_p": 0.75,
    "top_k": 40,
    "num_beams": 4,
    "max_new_tokens": 128
}

# Load the model
model = load_model(base_model)

# Generate a response
instruction = "Tell me about alpacas."
input_data = ""
response = generate(model, instruction, input_data, prompt_template, generation_params)
print(response)

In [ ]:
# Old testing code follows.

"""
# testing code for readme
for instruction in [
    "Tell me about alpacas.",
    "Tell me about the president of Mexico in 2019.",
    "Tell me about the king of France in 2019.",
    "List all Canadian provinces in alphabetical order.",
    "Write a Python program that prints the first 10 Fibonacci numbers.",
    "Write a program that prints the numbers from 1 to 100. But for multiples of three print 'Fizz' instead of the number and for the multiples of five print 'Buzz'. For numbers which are multiples of both three and five print 'FizzBuzz'.",  # noqa: E501
    "Tell me five words that rhyme with 'shock'.",
    "Translate the sentence 'I have no mouth but I must scream' into Spanish.",
    "Count up from 1 to 500.",
]:
    print("Instruction:", instruction)
    print("Response:", evaluate(instruction))
    print()
"""


# infer_rec

In [ ]:
import os
import sys
import json
# import fire  # Library for creating command line interfaces
import torch
import transformers
from peft import PeftModel  # Performance-efficient model wrapper
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
# from utils.prompter import Prompter  # Custom class for handling prompt generation
import pandas as pd
from tqdm import tqdm  # Library for progress bars

# Set environment variables for Transformers cache
os.environ['TRANSFORMERS_CACHE'] = '/data/private/peilin/cache'
os.environ['HF_HOME'] = '/data/private/peilin/cache'
# Uncomment to specify GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

# Check if CUDA (GPU support) is available and set the device accordingly
if torch.cuda.is_available():
    device = "cuda"

def load_instruction(instruct_dir):
    """
    Load instructions from a JSON file.

    Parameters:
    - instruct_dir (str): Path to the instruction JSON file.

    Returns:
    A list of instructions loaded from the file.
    """
    input_data = []
    with open(instruct_dir, "r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            d = json.loads(line)
            input_data.append(d)
    return input_data

def load_instruction_from_csv(instruct_dir, prompt_idx='all'):
    """
    Load instructions from a CSV file.

    Parameters:
    - instruct_dir (str): Path to the instruction CSV file.
    - prompt_idx (str): Specific prompt index to load. Defaults to 'all'.

    Returns:
    A tuple containing the list of instructions and the dataframe.
    """
    input_data = []
    df = pd.read_csv(instruct_dir, dtype='str', usecols=['prompt', 'target', 'task', 'few_zero'])
    dict_from_df = df.to_dict(orient='index')
    for key, value in dict_from_df.items():
        data = {
            'output': value['target'].strip(),
            'instruction': value['prompt'].strip()
        }
        input_data.append(data)
    return input_data, df

def main(
    # Parameters for the main function
    load_8bit: bool = False,
    base_model: str = "decapoda-research/llama-7b-hf",
    instruct_dir: str = "./data/beauty_sequential_single_prompt_test_sample.json",
    output_dir: str = "output/",
    use_lora: bool = False,
    lora_weights: str = "tloen/alpaca-lora-7b",
    prompt_template: str = "rec_template",
    max_new_tokens: int = 10,
    num_return_sequences: int = 10,
    num_beams: int = 10,
):
    """
    Main function for fine-tuning and evaluating a language model.

    Parameters:
    - load_8bit (bool): Flag to indicate if 8-bit precision is to be used.
    - base_model (str): Identifier for the base language model.
    - instruct_dir (str): Directory containing the instruction data.
    - output_dir (str): Directory to save output data.
    - use_lora (bool): Flag to indicate if LoRA weights are to be used.
    - lora_weights (str): Identifier for LoRA weights.
    - prompt_template (str): Template for generating prompts.
    - max_new_tokens (int): Maximum number of new tokens to generate.
    - num_return_sequences (int): Number of return sequences.
    - num_beams (int): Number of beams for beam search.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Initialize the prompter, tokenizer, and model
    prompter = Prompter(prompt_template)
    tokenizer = LlamaTokenizer.from_pretrained(base_model)
    model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,device_map="auto",)
    if use_lora:
    print(f"using lora {lora_weights}")
    model = PeftModel.from_pretrained(
    model,
    lora_weights,
    torch_dtype=torch.float16,
    )
    # Adjust the model configuration to handle specific token ids
    model.config.pad_token_id = tokenizer.pad_token_id = 0  # Set pad token id
    model.config.bos_token_id = 1  # Set beginning of sentence token id
    model.config.eos_token_id = 2  # Set end of sentence token id

    # Convert the model to half precision if 8-bit loading is not used
    if not load_8bit:
        model.half()  # This can fix bugs for some users

    model.eval()  # Set the model to evaluation mode

    # Compile the model with TorchScript for performance optimization
    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)

    def evaluate(
        instruction,
        input=None,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=num_beams,
        max_new_tokens=max_new_tokens,
        num_return_sequences=num_return_sequences,
        **kwargs,
    ):
        """
        Evaluate the model on a given instruction and input.

        Parameters:
        - instruction (str): Instruction or query to be processed.
        - input (str, optional): Additional input data.
        - temperature, top_p, top_k, num_beams, max_new_tokens, num_return_sequences: Model generation settings.

        Returns:
        A list of generated answers.
        """
        prompt = prompter.generate_prompt(instruction, input)
        inputs = tokenizer(prompt, return_tensors="pt")
        input_ids = inputs["input_ids"].to(device)
        generation_config = GenerationConfig(
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            num_beams=num_beams,
            **kwargs,
        )
        with torch.no_grad():
            generation_output = model.generate(
                input_ids=input_ids,
                generation_config=generation_config,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=max_new_tokens,
                num_return_sequences=num_return_sequences,
            )
        answer_list = []
        for i in range(num_return_sequences):
            s = generation_output.sequences[i]
            output = tokenizer.decode(s)
            answer_list.append(prompter.get_response(output))
        return '\t'.join(answer_list)

    def write_to_json(data, output_dir):
        """
        Write data to a JSON file.

        Parameters:
        - data: Data to be written.
        - output_dir (str): Path to the output JSON file.
        """
        with open(output_dir, 'w') as f:
            for item in data:
                json.dump(item, f)
                f.write('\n')

    def infer_from_json(instruct_dir):
        """
        Perform inference from a JSON file and save the results.

        Parameters:
        - instruct_dir (str): Path to the instruction JSON file.
        """
        input_data = load_instruction(instruct_dir)
        output_data = []
        for d in tqdm(input_data):
            instruction = d["source"]
            output = d["target"]
            model_output = evaluate(instruction)
            task_type = d["task_type"]
            output_data.append({'labels': d['target'], 'predict': model_output})
        file_name = "rec_{}_{}.json".format(task_type, base_model.split('/')[-1]) if not use_lora else 'rec_{}_{}_{}.json'.format(task_type, base_model.split('/')[-1], lora_weights.split('/')[-1])
        output_path = os.path.join(output_dir, file_name)
        write_to_json(output_data, output_path)

    # Conditional execution based on the provided instruction directory
    if instruct_dir != "":
        filename, file_extension = os.path.splitext(instruct_dir)
        if file_extension[1:] == 'json':
            infer_from_json(instruct_dir)
        else:
            raise ValueError("Unsupported file format")
    else:
        raise ValueError("Instruction directory not provided")

if name == "main":
fire.Fire(main)






# finetune_rec

In [ ]:
import os
import sys
from typing import List
import fire  # Library for creating command line interfaces
import torch
import transformers
from datasets import load_dataset  # Library for loading datasets

# Import LoRA related modules
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import LlamaForCausalLM, LlamaTokenizer
from utils.data_format_transform import filter_and_convert_rec


In [ ]:
def train(
    # Parameters for the train function
    base_model: str = "decapoda-research/llama-7b-hf",  # Base model identifier
    data_path: str = "data/beauty_sequential_single_prompt_train_sample.json",  # Training data path
    valid_data_path: str = "data/beauty_sequential_single_prompt_test.json",  # Validation data path
    output_dir: str = "./lora-alpaca",  # Output directory for the trained model
    # Training hyperparameters
    batch_size: int = 32,
    micro_batch_size: int = 2,
    num_epochs: int = 10,
    learning_rate: float = 3e-4,
    cutoff_len: int = 256,  # Max sequence length
    val_set_size: int = 500,  # Size of validation set
    save_eval_steps: int = 10,  # Steps interval for saving and evaluating
    # LoRA hyperparameters
    lora_r: int = 8,
    lora_alpha: int = 16,
    lora_dropout: float = 0.05,
    lora_target_modules: List[str] = ["q_proj", "v_proj"],  # Target modules for LoRA
    # LLM hyperparameters
    train_on_inputs: bool = False,  # If False, masks out inputs in loss calculation
    group_by_length: bool = False,  # Grouping strategy for efficiency
    # WandB (Weights & Biases) parameters for experiment tracking
    wandb_project: str = "llama_med",
    wandb_run_name: str = "",
    wandb_watch: str = "",
    wandb_log_model: str = "",
    resume_from_checkpoint: str = None,  # Path to resume training from a checkpoint
    prompt_template_name: str = "alpaca",  # Default prompt template
):
    """
    Trains the LlamaForCausalLM model with LoRA modifications.

    Parameters:
    - base_model (str): Identifier for the base pre-trained model.
    - data_path (str): Path to the training data file.
    - valid_data_path (str): Path to the validation data file.
    - output_dir (str): Directory to save the trained model and outputs.
    - Other parameters include various training settings, LoRA configurations, and WandB settings.
    """
    # Ensure only the primary process prints the training parameters
    if int(os.environ.get("LOCAL_RANK", 0)) == 0:
        # Printing training parameters for reference
        print(f"Training parameters: {locals()}")

    # Ensure the base model is specified
    assert base_model, "Base model must be specified."

    # Calculate the gradient accumulation steps
    gradient_accumulation_steps = batch_size // micro_batch_size

    # Initialize the prompter with the specified template
    prompter = Prompter(prompt_template_name)

    # Set up device mapping for distributed data parallel (DDP) if applicable
    device_map = "auto"
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
        gradient_accumulation_steps //= world_size

    # Determine if Weights & Biases (WandB) will be used for experiment tracking
    use_wandb = len(wandb_project) > 0 or (
        "WANDB_PROJECT" in os.environ and len(os.environ["WANDB_PROJECT"]) > 0
    )
    # Set WandB environment variables if specified
    if len(wandb_project) > 0:
        os.environ["WANDB_PROJECT"] = wandb_project
    if len(wandb_watch) > 0:
        os.environ["WANDB_WATCH"] = wandb_watch
    if len(wandb_log_model) > 0:
        os.environ["WANDB_LOG_MODEL"] = wandb_log_model

    # Load the base model
    model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map=device_map,
    )

    # Initialize the tokenizer
    tokenizer = LlamaTokenizer.from_pretrained(base_model)
    tokenizer.pad_token_id = 0  # Set pad token id to 'unk' (unknown)
    tokenizer.padding_side = "left"  # Set padding side for batched inference

    # Tokenization function
    def tokenize(prompt, add_eos_token=True):
        """
        Tokenizes the prompt with optional EOS token addition.
        """
        result = tokenizer(
            prompt,
            truncation=True,
            max_length=cutoff_len,
            padding=False,
            return_tensors=None,
        )
        # Add EOS token if needed and if space allows
        if (result["input_ids"][-1] != tokenizer.eos_token_id
                and len(result["input_ids"]) < cutoff_len
                and add_eos_token):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)

        # Set labels for training
        result["labels"] = result["input_ids"].copy()

        return result

    # Function to generate and tokenize prompts
    def generate_and_tokenize_prompt(data_point):
        """
        Generates a full prompt from the data point and tokenizes it.
        """
        full_prompt = prompter.generate_prompt(
            data_point["source"],
            None,
            data_point["target"],
        )
        tokenized_full_prompt = tokenize(full_prompt)
        if not train_on_inputs:
            user_prompt = prompter.generate_prompt(
                data_point["source"], None
            )
            tokenized_user_prompt = tokenize(user_prompt, add_eos_token=False)
            user_prompt_len = len(tokenized_user_prompt["input_ids"])

            tokenized_full_prompt["labels"] = [
                -100
            ] * user_prompt_len + tokenized_full_prompt["labels"][
                user_prompt_len:
            ]  # could be sped up, probably
        return tokenized_full_prompt

    # Prepare the model for int8 training (for efficient training)
    model = prepare_model_for_int8_training(model)

    # Configure LoRA settings
    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)

    # Load dataset
    if data_path.endswith(".json") or data_path.endswith(".jsonl"):
        data = load_dataset("json", data_files=data_path)
    else:
        data = load_dataset(data_path)

    if os.path.exists(valid_data_path):
        if valid_data_path.endswith(".json") or valid_data_path.endswith(".jsonl"):
            valid_data = load_dataset("json", data_files=valid_data_path)
        else:
            valid_data = load_dataset(data_path)
    else:
        valid_data = None

    if resume_from_checkpoint:
        # Check the available weights and load them
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "pytorch_model.bin"
        )  # Full checkpoint
        if not os.path.exists(checkpoint_name):
            checkpoint_name = os.path.join(
                resume_from_checkpoint, "adapter_model.bin"
            )  # only LoRA model - LoRA config above has to fit
            resume_from_checkpoint = (
                False  # So the trainer won't try loading its state
            )
        # The two files above have a different name depending on how they were saved, but are actually the same.
        if os.path.exists(checkpoint_name):
            print(f"Restarting from {checkpoint_name}")
            adapters_weights = torch.load(checkpoint_name)
            model = set_peft_model_state_dict(model, adapters_weights)
        else:
            print(f"Checkpoint {checkpoint_name} not found")

    model.print_trainable_parameters()  # Be more transparent about the % of trainable params.

    if val_set_size > 0 and not valid_data:
        train_val = data["train"].train_test_split(
            test_size=val_set_size, shuffle=True, seed=2023
        )
        train_data = (
            train_val["train"].shuffle().map(generate_and_tokenize_prompt)
        )
        val_data = (
            train_val["test"].shuffle().map(generate_and_tokenize_prompt)
        )
    elif val_set_size > 0 and valid_data:
        train_data = (
            data["train"].shuffle(seed=2023).map(generate_and_tokenize_prompt)
        )
        val_sample = valid_data["train"].train_test_split(
            test_size=val_set_size, shuffle=True, seed=2023
        )
        val_data = (
            val_sample["test"].shuffle().map(generate_and_tokenize_prompt)
        )
    else:
        train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
        val_data = None

    if not ddp and torch.cuda.device_count() > 1:
        # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
        model.is_parallelizable = True
        model.model_parallel = True

    # Set up the trainer
    # ... Trainer configuration and initialization

    trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=micro_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_ratio=0.1,
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            fp16=True,
            logging_steps=8,
            optim="adamw_torch",
            evaluation_strategy="steps" if val_set_size > 0 else "no",
            save_strategy="steps",
            eval_steps=save_eval_steps if val_set_size > 0 else None,
            save_steps=save_eval_steps,
            output_dir=output_dir,
            save_total_limit=1000,
            load_best_model_at_end=True if val_set_size > 0 else False,
            ddp_find_unused_parameters=False if ddp else None,
            group_by_length=group_by_length,
            report_to="wandb" if use_wandb else None,
            run_name=wandb_run_name if use_wandb else None,
        ),
        data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
    )
    model.config.use_cache = False


    # Adjust model's state_dict for PEFT
    old_state_dict = model.state_dict
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(
            self, old_state_dict()
        )
    ).__get__(model, type(model))

    # Compile the model with TorchScript for better performance if applicable
    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)

    # Start the training process
    trainer.train(resume_from_checkpoint=resume_from_checkpoint)

    # Save the trained model
    model.save_pretrained(output_dir)

    print("\nTraining completed successfully.")

if name == "main":
fire.Fire(train)




In [ ]:
def initialize_wandb(wandb_project, wandb_watch, wandb_log_model):
  """
  Sets up Weights & Biases (WandB) environment variables and configuration.
  """
  use_wandb = len(wandb_project) > 0 or (
    "WANDB_PROJECT" in os.environ and len(os.environ["WANDB_PROJECT"]) > 0
  )
  if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project
  if len(wandb_watch) > 0:
    os.environ["WANDB_WATCH"] = wandb_watch
  if len(wandb_log_model) > 0:
    os.environ["WANDB_LOG_MODEL"] = wandb_log_model
  return use_wandb

## load model

In [ ]:
def setup_model(base_model, config, wandb_project, wandb_watch, wandb_log_model):
    """
    Matches and sets up the device for the model, and loads the model accordingly and config it.
    """

    # Configures device mapping for distributed data parallel (DDP) if applicable.
    device_map = "auto"
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
        gradient_accumulation_steps //= world_size

    # Determine if Weights & Biases (WandB) will be used for experiment tracking
    use_wandb = initialize_wandb(wandb_project, wandb_watch, wandb_log_model)
    # Load the base model
    model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map=device_map,
    )

    # Prepare the model for int8 training (for efficient training)
    model = prepare_model_for_int8_training(model)

    # Configure LoRA settings
    model = get_peft_model(model, config)

    if resume_from_checkpoint:
    # Check the available weights and load them
    checkpoint_name = os.path.join(
        resume_from_checkpoint, "pytorch_model.bin"
    )  # Full checkpoint
    if not os.path.exists(checkpoint_name):
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "adapter_model.bin"
        )  # only LoRA model - LoRA config above has to fit
        resume_from_checkpoint = (
            False  # So the trainer won't try loading its state
        )
    # The two files above have a different name depending on how they were saved, but are actually the same.
    if os.path.exists(checkpoint_name):
        print(f"Restarting from {checkpoint_name}")
        adapters_weights = torch.load(checkpoint_name)
        model = set_peft_model_state_dict(model, adapters_weights)
    else:
        print(f"Checkpoint {checkpoint_name} not found")

    model.print_trainable_parameters()  # Be more transparent about the % of trainable params.

    if not ddp and torch.cuda.device_count() > 1:
      # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
      model.is_parallelizable = True
      model.model_parallel = True

    return model, use_wandb

## tokenize

In [ ]:
def initialize_tokenizer(base_model):
  """
  Initializes the tokenizer and sets pad token and padding side.
  """
  tokenizer = LlamaTokenizer.from_pretrained(base_model)
  tokenizer.pad_token_id = 0  # Set pad token id to 'unk' (unknown)
  tokenizer.padding_side = "left"  # Set padding side for batched inference
  return tokenizer

In [ ]:
# Tokenization function
def tokenize(prompt, tokenizer, cutoff_len, add_eos_token=True):
  """
  Tokenizes the prompt with optional EOS token addition.
  """
  result = tokenizer(
    prompt,
    truncation=True,
    max_length=cutoff_len,
    padding=False,
    return_tensors=None,
  )
  # Add EOS token if needed and if space allows
  if (result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token):
    result["input_ids"].append(tokenizer.eos_token_id)
    result["attention_mask"].append(1)

  # Set labels for training
  result["labels"] = result["input_ids"].copy()

  return result

In [ ]:
# Function to generate and tokenize prompts
def generate_and_tokenize_prompt(data_point, train_on_inputs, prompter, cutoff_len, tokenizer):
  """
  Generates a full prompt from the data point and tokenizes it.
  """
  full_prompt = prompter.generate_prompt(
    data_point["source"],
    None,
    data_point["target"],
  )
  tokenized_full_prompt = tokenize(full_prompt, cutoff_len=cutoff_len, tokenizer=tokenizer)
  if not train_on_inputs:
    user_prompt = prompter.generate_prompt(
      data_point["source"], None
    )
    tokenized_user_prompt = tokenize(user_prompt, cutoff_len=cutoff_len,
                                     tokenizer=tokenizer, add_eos_token=False)
    user_prompt_len = len(tokenized_user_prompt["input_ids"])

    tokenized_full_prompt["labels"] = [
      -100
    ] * user_prompt_len + tokenized_full_prompt["labels"][
      user_prompt_len:
    ]  # could be sped up, probably
  return tokenized_full_prompt

## load data

In [ ]:
def setup_dataset(data_path, valid_data_path, val_set_size, prompter, cutoff_len, tokenizer, train_on_inputs):
    """
    Load training and validation datasets.

    Args:
        data_path (str): Path to the training data file.
        valid_data_path (str): Path to the validation data file.
        val_set_size (int): Size of the validation set.
        generate_and_tokenize_prompt (function): Function to generate and tokenize prompts.

    Returns:
        tuple: Tuple containing the training and validation datasets.
    """
    generate_and_tokenize_prompt = lambda data_point: generate_and_tokenize_prompt(data_point, train_on_inputs=train_on_inputs, cutoff_len=cutoff_len, prompter=prompter, tokenizer=tokenizer)
    # Helper function to load dataset based on file extension
    def load_data(path):
        if path.endswith(".json") or path.endswith(".jsonl"):
            return load_dataset("json", data_files=path)
        return load_dataset(path)

    # Load datasets
    data = load_data(data_path)
    valid_data = load_data(valid_data_path) if os.path.exists(valid_data_path) else None

    # Split and prepare datasets
    if val_set_size > 0:
        if valid_data:
            train_data = data["train"].shuffle(seed=2023).map(generate_and_tokenize_prompt)
            val_data = valid_data["train"].train_test_split(
                test_size=val_set_size, shuffle=True, seed=2023
            )["test"].shuffle().map(generate_and_tokenize_prompt)
        else:
            train_val = data["train"].train_test_split(
                test_size=val_set_size, shuffle=True, seed=2023
            )
            train_data = train_val["train"].shuffle().map(generate_and_tokenize_prompt)
            val_data = train_val["test"].shuffle().map(generate_and_tokenize_prompt)
    else:
        train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
        val_data = None

    return train_data, val_data

## train model

In [ ]:
# import pandas as pd

# d = {'x':[1,2,3], 'y':[4,5,6]}
# df = pd.DataFrame(d)
# df['x'].map(lambda x: f(x, y=5))

TypeError: f() got an unexpected keyword argument 'y'

In [ ]:
# def f(x):
#   x = [1,2,3]
#   y = [2]
#   return y
# x = [7]
# y = f(x)
# print(x)

[7]


In [ ]:
# Main training process function
def train_model(
    # Parameters for the train function
    base_model: str = "decapoda-research/llama-7b-hf",  # Base model identifier
    data_path: str = "data/beauty_sequential_single_prompt_train_sample.json",  # Training data path
    valid_data_path: str = "data/beauty_sequential_single_prompt_test.json",  # Validation data path
    output_dir: str = "./lora-alpaca",  # Output directory for the trained model
    # Training hyperparameters
    batch_size: int = 32,
    micro_batch_size: int = 2,
    num_epochs: int = 10,
    learning_rate: float = 3e-4,
    cutoff_len: int = 256,  # Max sequence length
    val_set_size: int = 500,  # Size of validation set
    save_eval_steps: int = 10,  # Steps interval for saving and evaluating
    # LoRA hyperparameters
    lora_r: int = 8,
    lora_alpha: int = 16,
    lora_dropout: float = 0.05,
    lora_target_modules: list[str] = ["q_proj", "v_proj"],  # Target modules for LoRA
    # LLM hyperparameters
    train_on_inputs: bool = False,  # If False, masks out inputs in loss calculation
    group_by_length: bool = False,  # Grouping strategy for efficiency
    # WandB (Weights & Biases) parameters for experiment tracking
    wandb_project: str = "llama_med",
    wandb_run_name: str = "",
    wandb_watch: str = "",
    wandb_log_model: str = "",
    resume_from_checkpoint: str = None,  # Path to resume training from a checkpoint
    prompt_template_name: str = "alpaca",  # Default prompt template
    ):
    """
    Orchestrates the entire training process.
    """
    # Ensure only the primary process prints the training parameters
    if int(os.environ.get("LOCAL_RANK", 0)) == 0:
        # Printing training parameters for reference
        print(f"Training parameters: {locals()}")

    # Ensure the base model is specified
    assert base_model, "Base model must be specified."

    # Calculate the gradient accumulation steps
    gradient_accumulation_steps = batch_size // micro_batch_size

    #load model and setup config its
    lora_config = LoraConfig(r=lora_r, lora_alpha=lora_alpha,# Configure LoRA settings
                             target_modules=lora_target_modules, lora_dropout=lora_dropout)
    model, use_wandb = setup_model(base_model=base_model, config=lora_config,
                       wandb_project=wandb_project, wandb_watch=wandb_watch,
                                  wandb_log_model=wandb_log_model)

    # Initialize the prompter with the specified template
    prompter = Prompter(prompt_template_name)

    tokenizer = initialize_tokenizer(base_model)

    # setup data
    train_data, val_data = setup_dataset(data_path=data_path, valid_data_path=valid_data_path, val_set_size=val_set_size,
                                         prompter=prompter, cutoff_len=cutoff_len, tokenizer=tokenizer, train_on_inputs=train_on_inputs)

    # Set up the trainer
    # ... Trainer configuration and initialization

    trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=micro_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_ratio=0.1,
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            fp16=True,
            logging_steps=8,
            optim="adamw_torch",
            evaluation_strategy="steps" if val_set_size > 0 else "no",
            save_strategy="steps",
            eval_steps=save_eval_steps if val_set_size > 0 else None,
            save_steps=save_eval_steps,
            output_dir=output_dir,
            save_total_limit=1000,
            load_best_model_at_end=True if val_set_size > 0 else False,
            ddp_find_unused_parameters=False if ddp else None,
            group_by_length=group_by_length,
            report_to="wandb" if use_wandb else None,
            run_name=wandb_run_name if use_wandb else None,
        ),
        data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
    )
    model.config.use_cache = False


    # Adjust model's state_dict for PEFT
    old_state_dict = model.state_dict
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(
            self, old_state_dict()
        )
    ).__get__(model, type(model))

    # Compile the model with TorchScript for better performance if applicable
    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)

    # Start the training process
    trainer.train(resume_from_checkpoint=resume_from_checkpoint)

    # Save the trained model
    model.save_pretrained(output_dir)

    print("\nTraining completed successfully.")

In [ ]:
def main_train():
  pass